In [17]:
import pandas as pd
import requests

def get_headers():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/90.0.4430.212 Safari/537.36'
    }
    return headers

class get_11v11:
    def __init__(self, date, venue = None):
        self.date = date
        self.venue = venue
        
        self.url = self.get_url()

        r = requests.get(self.url, headers = get_headers())

        self.tables = pd.read_html(r.content, flavor="bs4")
        
        self.table = self.get_table()
        self.pos = self.get_pos()

    def get_url(self):
        year = self.date[:4]
        month = pd.to_datetime(self.date).month_name().lower()
        day = self.date[8:]

        url = f"https://www.11v11.com/league-tables/league-two/{day}-{month}-{year}/"
        if self.venue and self.venue[0].upper() in ["H", "A"]:
            if self.venue[0].upper() == "H":
                self.venue = "home"
            else:
                self.venue = "away"
            url += f"{self.venue}"
        return url
    
    def get_table(self):
        table = self.tables[0]
        table.Pos = table.Pos.index + 1
        return table
    
    def get_pos(self):
        try:
            pos = self.table.query("Team.str.contains('Tranmere')").Pos.values[0]
        except:
            pos = "No table containing Tranmere Rovers found"
        return pos